## Подготовка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from math import sqrt
import xgboost as xg
import lightgbm as lgb
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.info()

In [ ]:
df.columns

Удалим столбцы, не нужные для обучения - это столбцы с информацией о дате скачивания анкеты из базы, дате создания анкеты, количестве информации автомобиля, почтовом индексом владельца анкеты и дате последней активности пользователя

In [ ]:
df = df.drop(['DateCrawled','RegistrationMonth','DateCreated', 'NumberOfPictures','PostalCode','LastSeen'], axis = 1)

In [ ]:
df.head(10)

Рассмотрим уникальные значения в категориальных признаках

In [ ]:
df_categorical = ['VehicleType','Gearbox', 'Model',
       'FuelType', 'Brand']

In [ ]:
for i in df_categorical:
    print(i)
    print(df[i].unique())
    print()

Уникальные значения признаков 'Brand' и 'Model' слишком многочислены, да и один бренд может иметь как дорогие, так и дешевые модели автомобилей, поэтому данные признаки не являются показательными. Во избежание длительного обучения, удалим данные столбцы.

In [ ]:
df = df.drop(['Brand','Model'], axis = 1)

Теперь избавимся от явных дубликатов и пропущенных значений 

In [ ]:
df.isna().sum()/df.isna().count()

Пропуски в столбце "Repaired" предположительно означают, что машина не была отремонтирована перед выставлением на продажу, поэтому их заполним значением "no"

In [ ]:
df.loc[df['Repaired'].isna() == True, 'Repaired'] = 'no'

Пропуски в столбцах 'FuelType', 'VehicleType' и 'Gearbox' заполним модами

In [ ]:
df.loc[df['FuelType'].isna() == True, 'FuelType'] = df['FuelType'].mode()[0]
df.loc[df['Gearbox'].isna() == True, 'Gearbox'] = df['Gearbox'].mode()[0]
df.loc[df['VehicleType'].isna() == True, 'VehicleType'] = df['VehicleType'].mode()[0]

In [ ]:
df.isna().sum()/df.isna().count()

In [ ]:
df.info()

In [ ]:
df['RegistrationYear'].plot( kind = 'box',figsize = (5, 5))
plt.ylim(1900,2100)

In [ ]:
df = df.query('RegistrationYear > 1980 and RegistrationYear < 2022')

In [ ]:
df['Price'].plot( kind = 'box',figsize = (5, 5))


In [ ]:
df = df.query('Price > 300 and Price < 15000')

In [ ]:
df['Power'].plot( kind = 'box',figsize = (5, 5))
plt.ylim(0,1000)

In [ ]:
df = df.query('Power > 70 and Power < 500')

In [ ]:
np.sort(df['Kilometer'].unique())

Разделим выборку на тренировочную, валидационную и тестовую

In [ ]:
df_target = df['Price']
df_features = df.drop('Price', axis = 1)

In [ ]:
df_temp_features, df_test_features, df_temp_target, df_test_target = train_test_split( df_features, df_target, test_size = 0.2)
df_train_features, df_valid_features, df_train_target, df_valid_target = train_test_split( df_temp_features, df_temp_target,test_size = 0.25)

Категориальные признаки нужно закодировать, а количественные стандартизировать. Для кодирования данных для модели линейной регрессии будем использовать OHE, для остальных - OrdinalEncoder.

In [ ]:
ohe_encoder = OneHotEncoder(drop='first')
var_categorical = ['VehicleType','Gearbox','FuelType','Repaired']
ohe_encoder.fit(df_train_features[var_categorical ])

def ohe_encode(features, encoder):
    arr = encoder.transform(features[var_categorical]).toarray()
    col = []
    for i in encoder.categories_:
        c = 0
        for j in i:
            if c == 0:
                c += 1
            else:
                col.append(j)
    features_ohe = pd.DataFrame(arr, columns = col)
    features_ohe = features_ohe.astype(int)
    return features_ohe
    
features_train_ohe = ohe_encode(df_train_features, ohe_encoder)
features_test_ohe = ohe_encode(df_test_features, ohe_encoder)
features_valid_ohe = ohe_encode(df_valid_features, ohe_encoder)

In [ ]:
oe_encoder = OrdinalEncoder()
var_categorical = ['VehicleType','Gearbox','FuelType','Repaired']
oe_encoder.fit(df_train_features[var_categorical])

def oe_encode(features, encoder):
    arr = oe_encoder.transform(features[var_categorical])
    features_oe = pd.DataFrame(data = arr, columns = var_categorical)
    features_oe = features_oe.astype(int)
    return features_oe

features_train_oe = oe_encode(df_train_features, oe_encoder)
features_test_oe = oe_encode(df_test_features, oe_encoder)
features_valid_oe = oe_encode(df_valid_features, oe_encoder)

In [ ]:
scaler = StandardScaler()
var_countable = ['RegistrationYear','Kilometer','Power']
scaler.fit(df_train_features[var_countable])
def scale(features, scaler):
    arr = scaler.transform(features[var_countable])
    data = pd.DataFrame(data = arr, columns = var_countable)
    return data


features_train_st = scale(df_train_features, scaler)
features_test_st = scale(df_test_features, scaler)
features_valid_st = scale(df_valid_features, scaler)

In [ ]:
features_train_oe = pd.concat([features_train_oe, features_train_st], axis = 1)
features_test_oe = pd.concat([features_test_oe, features_test_st], axis = 1)
features_valid_oe = pd.concat([features_valid_oe, features_valid_st], axis = 1)

features_train_ohe = pd.concat([features_train_ohe, features_train_st], axis = 1)
features_test_ohe = pd.concat([features_test_ohe, features_test_st], axis = 1)
features_valid_ohe = pd.concat([features_valid_ohe, features_valid_st], axis = 1)

## Обучение моделей

LIGHTGBM Regressor:

In [ ]:
%%timeit
model = lgb.LGBMRegressor()
model.fit(features_train_oe, df_train_target)

Модель обучается за 3.7 +- 1.6 секунд 

In [ ]:
model = lgb.LGBMRegressor()
grid_space = {'boosting_type':['gbdt', 'dart', 'rf'],'num_leaves':range(20,41,10), 'n_estimators':range(100, 161,20)}
grid = GridSearchCV(model, param_grid = grid_space, cv=2, scoring = 'neg_root_mean_squared_error', n_jobs = -1 )

In [ ]:
model_grid = grid.fit(features_train_oe, df_train_target)

print(model_grid.best_params_)
print('RMSE:', model_grid.best_score_)

{'boosting_type': 'gbdt', 'n_estimators': 160, 'num_leaves': 40}
RMSE: -1999.5516975956489
3min 20s ± 1.58 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

В результате подбора гиперпараметров и обучения модель бустинга LIGHTGBM показала значение метрики RMSE по модулю в 1999.555, что удовлетворяет условию заказчика.

Проверка на валидационной выборке:

In [ ]:
model = lgb.LGBMRegressor(boosting_type = 'gbdt', n_estimators = 160, num_leaves =40)
model.fit(features_train_oe, df_train_target)

In [ ]:
%%timeit
pred = model.predict(features_valid_oe)
print(sqrt(MSE(df_valid_target, pred)))

Время предсказания модели - 642 мс +- 305 мс

XGB Regressor:


In [ ]:
%%timeit
model_1 = xg.XGBRegressor()
model_1.fit(features_train_oe, df_train_target)

Модель обучается за 11 с +- 1 с 

In [ ]:
model_1 = xg.XGBRegressor()
grid_space_1 = {'max_depth':range(5,10), 'n_estimators':(1,10), 'max_leaves':(10,50)}
grid_1 = GridSearchCV(model_1, param_grid = grid_space_1, cv = 5, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

In [ ]:
model_grid_1 = grid_1.fit(features_train_oe, df_train_target)
print(model_grid_1.best_params_)
print('RMSE:', model_grid_1.best_score_)

{'max_depth': 9, 'max_leaves': 10, 'n_estimators': 10}
RMSE: -2032.0607909695304
1min 37s ± 1.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

В результате обучения XGB регрессора было достигнуто значения метрики RMSE по модулю в 2030, что не сильно уступает значению предудущей модели. По времени обучения XGB Regressor уступает предыдущей модели.

Проверка на валидационной выборке:

In [ ]:
model_1 = xg.XGBRegressor(max_depth = 9, max_leaves = 10, n_estimators =  10)
model_1.fit(features_train_oe, df_train_target)

In [ ]:
%%timeit
pred_1 = model_1.predict(features_valid_oe)
print(sqrt(MSE(df_valid_target, pred_1)))

Время предсказания модели - 95 мс +- 9.6 мс

Линейная регрессия:

In [ ]:
%%timeit
model_2 = LinearRegression()
model_2.fit(features_train_ohe, df_train_target)

Модель обучается за 293 мс ± 41.9 мс

In [ ]:
model_2 = LinearRegression()
grid_space_2 = {'fit_intercept':[True, False], 'copy_X':[True, False], 'positive':[True, False],'n_jobs':[-1, 1]}
grid_2 = GridSearchCV(model_2, param_grid = grid_space_2, cv = 5, scoring='neg_root_mean_squared_error', n_jobs = -1)

In [ ]:
%%timeit
model_grid_2 = grid_2.fit(features_train_ohe, df_train_target)

print(model_grid_2.best_params_)
print('RMSE:', model_grid_2.best_score_)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': False}
RMSE: -3597.557531282143
17.9 s ± 2.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Значение метрики RMSE линейной регрессии составило 3597.55 по модулю, что не удовлетворяет требованиям заказчика. Обучалась модель 293 мс. Проверку на валидационной выборке смысла делать нет.

## Анализ моделей


Наилучшее значение метрики RMSE показала модель LightGBM Regressor. XGB Regressor несильно уступает в точности предсказания, но обучается медленне. Однако, предказывает данная модель быстрее, чем LightGBM Regressor.  Линейная регрессия, хоть и тратит меньше всего времени на обучение, выдает неудовлетворительную RMSE. В итоге, для предсказания рекомендую использовать именно LightGBM Regressor. Проверим его на тестовой выборке. 


In [ ]:
model_final = lgb.LGBMRegressor(boosting_type = 'gbdt', n_estimators = 160, num_leaves =40)
model_final.fit(features_train_oe, df_train_target)
pred_final = model_final.predict(features_test_oe)

print(sqrt(MSE(df_test_target, pred_final)))

Проверка на тестовой выборке увенчалась успехом - значение метрики RMSE удовлетворяет условию.